In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import catboost as cat

In [ ]:
data = pd.read_pickle('matrix.pickle')
X_train = data[data.date_block_num < 33].drop(['date_block_num','item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['date_block_num','item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['date_block_num','item_cnt_month'], axis=1)
del data
gc.collect();

In [ ]:
m2 = cat.CatBoostRegressor(
    #iterations=100, 
    #learning_rate=0.1, 
    #depth=11, 
    loss_function='RMSE', 
    eval_metric='RMSE', 
    random_seed=42, 
    od_type='Iter', 
    od_wait=20,
    allow_writing_files=False) 

In [ ]:
ts = time.time()
categorical_features_indices = cat_features=[0,1,2,3,4,5,6]
train_pool = cat.Pool(X_train, Y_train, cat_features=categorical_features_indices)
validation_pool = cat.Pool(X_valid, Y_valid, cat_features=categorical_features_indices)
time.time()- ts

In [ ]:
ts = time.time()
m2.fit(train_pool, eval_set=validation_pool, use_best_model=True, verbose=True)
time.time()- ts

In [ ]:
feature_importances = m2.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
test = pd.read_csv('test.csv.gz', compression='gzip').set_index('ID')
Y_test = m2.predict(X_test).clip(0, 20)
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)

In [ ]:
m2.save_model('cat.model', format="cbm")